In [1]:
import cv2
import Detect_inator
from detect import *
import Model_structure
import model
import train

2024-11-10 12:14:13.038961: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 12:14:13.073124: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731233653.099697   20857 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731233653.106550   20857 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 12:14:13.127023: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [9]:
train.train_yolo(
    model_path="dataset/weights/yolov3.cfg",
    data_path="dataset/train/images",
    epochs=100,
    batch_size=16,
    pretrained_weights="dataset/weights/yolov3.weights",
    checkpoint_interval=5,
    evaluation_interval=5,
    multiscale_training=True,
    seed=42,
    verbose=True
)

Environment information:
System: Linux 5.15.153.1-microsoft-standard-WSL2
Not using the poetry package
Current Commit Hash: 3d68ccb


IsADirectoryError: [Errno 21] Is a directory: 'dataset/train/images'

In [2]:
import os
import pandas as pd

# Define paths
dataset_path = 'dataset'
splits = ['train', 'test', 'valid']

for split in splits: 
    # Path to the CSV file for each split
    csv_path = os.path.join(dataset_path, split, '_annotations.csv')
    images_folder = os.path.join(dataset_path, split, 'images')
    labels_folder = os.path.join(dataset_path, split, 'labels')

    # Create labels folder if it doesn't exist
    os.makedirs(labels_folder, exist_ok=True)

    # Load CSV
    data = pd.read_csv(csv_path)

    # Group data by filename to handle multiple boxes per image
    grouped = data.groupby('filename')

    for filename, group in grouped:
        # Get image dimensions from the first row (assuming they are the same for all boxes of the same image)
        width = group.iloc[0]['width']
        height = group.iloc[0]['height']
        
        # Prepare the label file path
        label_filename = os.path.splitext(filename)[0] + '.txt'
        label_path = os.path.join(labels_folder, label_filename)

        # Write bounding boxes to the label file
        with open(label_path, 'w') as f:
            for _, row in group.iterrows():
                class_id = row['class']  # Assuming 'player' is mapped to 0 if it's the only class
                xmin = row['xmin']
                ymin = row['ymin']
                xmax = row['xmax']
                ymax = row['ymax']
                
                # Convert to YOLO format
                x_center = ((xmin + xmax) / 2) / width
                y_center = ((ymin + ymax) / 2) / height
                box_width = (xmax - xmin) / width
                box_height = (ymax - ymin) / height

                # Write in the format: class_id x_center y_center box_width box_height
                f.write(f"0 {x_center} {y_center} {box_width} {box_height}\n")


In [5]:
import os
from glob import glob

# Define the root path to your dataset
dataset_root = "dataset"

# Define paths for each dataset split
splits = ["train", "valid", "test"]

# Function to create .txt files for each split
def create_txt_files(dataset_root, splits):
    for split in splits:
        # Path to the images folder
        images_path = os.path.join(dataset_root, split, "images")
        
        # Path to save the .txt file
        txt_file_path = os.path.join(dataset_root, f"{split}.txt")
        
        # Get list of all image files (adjust extension if necessary)
        image_files = glob(os.path.join(images_path, "*.jpg")) + glob(os.path.join(images_path, "*.png"))
        
        # Write image file paths to .txt file
        with open(txt_file_path, "w") as file:
            for image_path in image_files:
                file.write(f"{image_path}\n")
        
        print(f"{split}.txt created with {len(image_files)} entries at {txt_file_path}")

# Run the function
create_txt_files(dataset_root, splits)

train.txt created with 106 entries at dataset/train.txt
valid.txt created with 68 entries at dataset/valid.txt
test.txt created with 32 entries at dataset/test.txt


In [2]:
model_my = model.load_model('dataset/weights/yolov3.cfg', 'dataset/weights/yolov3_custom.weights')

In [4]:
img = cv2.imread('dataset/train/images/rally_women_mp4-2_jpg.rf.b42f001028679c9a819540f3d497cc0c.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [5]:
boxes = detect_image(model_my, img)
print(boxes)

[]


In [5]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
test_data = pd.read_csv('dataset/train/_annotations.csv')

In [4]:
test_data

,filename,width,height,class,xmin,ymin,xmax,ymax
0,volley-76-_png.rf.055cd0a41516a2b5d3a10e8a321f...,640,640,player,74,138,110,284
1,volley-76-_png.rf.055cd0a41516a2b5d3a10e8a321f...,640,640,player,201,140,232,317
2,volley-76-_png.rf.055cd0a41516a2b5d3a10e8a321f...,640,640,player,150,137,204,361
3,volley-76-_png.rf.055cd0a41516a2b5d3a10e8a321f...,640,640,player,0,128,46,380
4,volley-76-_png.rf.055cd0a41516a2b5d3a10e8a321f...,640,640,player,196,197,265,376
...,...,...,...,...,...,...,...,...
1015,volley-103-_png.rf.f5d1869da6b094a1e9639de52b1...,640,640,player,592,288,640,431
1016,volley-103-_png.rf.f5d1869da6b094a1e9639de52b1...,640,640,player,105,327,159,478
1017,volley-103-_png.rf.f5d1869da6b094a1e9639de52b1...,640,640,player,474,334,508,519
1018,volley-103-_png.rf.f5d1869da6b094a1e9639de52b1...,640,640,player,389,357,448,521


In [11]:
test_data.query('filename == "rally_women_mp4-1_jpg.rf.9358bbdf94234961674ace645202858f.jpg"')

,filename,width,height,class,xmin,ymin,xmax,ymax
484,rally_women_mp4-1_jpg.rf.9358bbdf94234961674ac...,640,640,player,195,351,215,406
485,rally_women_mp4-1_jpg.rf.9358bbdf94234961674ac...,640,640,player,187,386,210,469
486,rally_women_mp4-1_jpg.rf.9358bbdf94234961674ac...,640,640,player,212,401,232,485
487,rally_women_mp4-1_jpg.rf.9358bbdf94234961674ac...,640,640,player,249,381,273,455
488,rally_women_mp4-1_jpg.rf.9358bbdf94234961674ac...,640,640,player,263,394,281,477
489,rally_women_mp4-1_jpg.rf.9358bbdf94234961674ac...,640,640,player,329,385,341,470
490,rally_women_mp4-1_jpg.rf.9358bbdf94234961674ac...,640,640,player,331,351,344,427
491,rally_women_mp4-1_jpg.rf.9358bbdf94234961674ac...,640,640,player,341,359,357,450
492,rally_women_mp4-1_jpg.rf.9358bbdf94234961674ac...,640,640,player,380,342,402,418
493,rally_women_mp4-1_jpg.rf.9358bbdf94234961674ac...,640,640,player,436,408,455,489
